<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/7.%20AdvancedTopics/5.%20%40property.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

7. AdvancedTopics/5. @property.ipynb

**@property**

Python has a great concept called property which makes the life of an object oriented programmer much simpler.

Before defining and going into details of what @property is, let us first build an intuition on why it would be needed in the first place.

An Example To Begin With

Let us assume that you decide to make a class that could store the temperature in degree Celsius. It would also implement a method to convert the temperature into degree Fahrenheit. One way of doing this is as follows.



In [0]:
class Celsius:
    def __init__(self, temperature = 0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

We could make objects out of this class and manipulate the attribute temperature as we wished. Try these on Python shell.

In [0]:
# create new object
man = Celsius()

In [0]:
# set temperature
man.temperature = 37

In [4]:
# get temperature
man.temperature

37

In [5]:
# get degrees Fahrenheit
man.to_fahrenheit()

98.60000000000001


The extra decimal places when converting into Fahrenheit is due to the floating point arithmetic error (try 1.1 + 2.2 in the Python interpreter).

Whenever we assign or retrieve any object attribute like temperature, as show above, Python searches it in the object's **\_\_dict__** dictionary.

In [6]:
man.__dict__

{'temperature': 37}

Therefore, man.temperature internally becomes man.\_\_dict__['temperature'].

Now, let's further assume that our class got popular among clients and they started using it in their programs. They did all kinds of assignments to the object.

One fateful day, a trusted client came to us and suggested that temperatures cannot go below -273 degree Celsius (students of thermodynamics might argue that it's actually -273.15), also called the absolute zero. He further asked us to implement this value constraint. Being a company that strive for customer satisfaction, we happily heeded the suggestion and released version 1.01 (an upgrade of our existing class).

**Using Getters and Setters**

An obvious solution to the above constraint will be to hide the attribute temperature (make it private) and define new getter and setter interfaces to manipulate it. This can be done as follows.


In [0]:
class Celsius:
    def __init__(self, temperature = 0):
        self.set_temperature(temperature)

    def to_fahrenheit(self):
        return (self.get_temperature() * 1.8) + 32

    # new update
    def get_temperature(self):
        return self._temperature

    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        self._temperature = value

We can see above that new methods get_temperature() and set_temperature() were defined and furthermore, temperature was replaced with _temperature. An underscore (_) at the beginning is used to denote private variables in Python

In [8]:
c = Celsius(-277)

ValueError: ignored

In [0]:
c = Celsius(37)

In [10]:
c.get_temperature()

37

In [11]:
c.set_temperature(-300)

ValueError: ignored

This update successfully implemented the new restriction. We are no longer allowed to set temperature below -273.

Please note that private variables don't exist in Python. There are simply norms to be followed. The language itself don't apply any restrictions.

In [0]:
c._temperature = -300

In [13]:
 c.get_temperature()

-300

But this is not of great concern. The big problem with the above update is that, all the clients who implemented our previous class in their program have to modify their code from obj.temperature to obj.get_temperature() and all assignments like obj.temperature = val to obj.set_temperature(val).

This refactoring can cause headaches to the clients with hundreds of thousands of lines of codes.

All in all, our new update was not backward compatible. This is where property comes to rescue.

**The Power of @property**

The pythonic way to deal with the above problem is to use property. Here is how we could have achieved it.


In [0]:
class Celsius:
    def __init__(self, temperature = 0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    def get_temperature(self):
        print("Getting value")
        return self._temperature

    def set_temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value

    temperature = property(get_temperature,set_temperature)

And, issue the following code in shell once you run it.


In [15]:
c = Celsius()

Setting value


We added a **print**() function inside **get_temperature**() and **set_temperature**() to clearly observe that they are being executed.

The last line of the code, makes a property object **temperature**. Simply put, **property** attaches some code (**get_temperature** and **set_temperature**) to the member attribute accesses (temperature).

Any code that retrieves the value of **temperature** will automatically call **get_temperature**() instead of a dictionary (\_\_dict__) look-up. Similarly, any code that assigns a value to **temperature** will automatically call **set_temperature**(). This is one cool feature in Python.

We can see above that **set_temperature**() was called even when we created an object.


**Can you guess why?**

The reason is that when an object is created, \_\_init__() method gets called. This method has the line self.temperature = temperature. This assignment automatically called set_temperature().

In [16]:
c.temperature

Getting value


0

Similarly, any access like c.temperature automatically calls get_temperature(). This is what property does. Here are a few more examples.

In [17]:
c.temperature = 37

Setting value


In [18]:
c.to_fahrenheit()

Getting value


98.60000000000001

By using property, we can see that, we modified our class and implemented the value constraint without any change required to the client code. Thus our implementation was backward compatible and everybody is happy.

Finally note that, the actual temperature value is stored in the private variable _temperature. The attribute temperature is a property object which provides interface to this private variable.